In [40]:
import requests
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import sqlite3

### Purple Air's API

In [41]:
url_groupdetail = "https://api.purpleair.com/v1/groups/1662"
url_newmember = "https://api.purpleair.com/v1/groups/1662/members"
url_sensordata = "https://api.purpleair.com/v1/sensors/39355"
url_groupdata = "https://api.purpleair.com/v1/groups/1662/members"
url_deletemember = "https://api.purpleair.com/v1/groups/1662/members/158862"

key_get = "C111944A-B3A7-11ED-B6F4-42010A800007"
key_post = "C112A231-B3A7-11ED-B6F4-42010A800007"
headers = {'X-API-Key': key_get}

### Create Sensor's database

In [42]:
# Connect to database.

conn = sqlite3.connect('air-pollution.db')
c = conn.cursor()
c.execute("PRAGMA foreign_keys = ON;")

In [43]:
# Create the "sensors" table.

c.execute('''CREATE TABLE sensors 
    ([sensor_id] INTEGER PRIMARY KEY, [municipio] TEXT, [estado] TEXT)''')

In [44]:
# Insert sensors.

c.execute(''' INSERT INTO sensors VALUES
        (34641, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37747, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (38537, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (143372, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (39355, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (39697, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (34569, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (39731, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (147010, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (140440, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (173083, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (45769, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (121825, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (34511, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (49563, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37657, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (50555, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (39725, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37861, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (50565, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37643, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (39733, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (140432, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37253, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (34499, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (118909, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37661, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (36709, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (33117, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (35011, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (34969, "San Pedro Garza García", "Nuevo Leon")''')

c.execute(''' INSERT INTO sensors VALUES
        (37255, "San Pedro Garza García", "Nuevo Leon")''')

In [45]:
# Create the "air_pollution" table.

c.execute('''CREATE TABLE air_pollution
        ([pollution_id] INTEGER PRIMARY KEY, [sensor_id] INTEGER, [pm25] REAL, [pm10] REAL, [temp_celsius] REAL, [date] TEXT,
        FOREIGN KEY (sensor_id) REFERENCES sensors (sensor_id)) ''')

In [50]:
# Insert data into "air_pollution" table.

sensors = [34641, 37747, 38537, 143372, 39355, 39697, 34569, 39731, 147010, 140440, 173083, 45769, 121825, 34511,
          49563, 37657, 50555, 39725, 37861, 50565, 37643, 39733, 140432, 37253, 34499, 118909, 37661, 36709,
          33117, 35011, 34969, 37255]

for sensor in sensors:
    
    url = f"https://api.purpleair.com/v1/sensors/{sensor}"
    
    sensor_id = sensor
    
    payload_pm25 = {'fields': 'pm2.5'}
    payload_pm10 = {'fields': 'pm10.0'}
    payload_temp = {'fields': 'temperature'}

    pm25 = requests.get(url, headers = headers, params = payload_pm25)
    pm25 = pm25.json()
    pm25 = pm25['sensor']['pm2.5']

    pm10 = requests.get(url, headers = headers, params = payload_pm10)
    pm10 = pm10.json()
    pm10 = pm10['sensor']['pm10.0']

    temp = requests.get(url, headers = headers, params = payload_temp)
    temp = temp.json()
    temp = temp['sensor']['temperature']
    temp_celsius = np.round((temp - 32) * .5556, decimals = 0)
    
    date = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

    c.execute('''INSERT INTO air_pollution
        (sensor_id, pm25, pm10, temp_celsius, date) VALUES(?, ?, ?, ?, ?)''',
        (sensor_id, pm25, pm10, temp_celsius, date))
    
    conn.commit()
    
conn.close()

In [51]:
# Fetch data from air_pollution
conn = sqlite3.connect('air-pollution.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          pollution_id, sensor_id, pm25, pm10, temp_celsius, date FROM air_pollution;
          ''')

df = pd.DataFrame(c.fetchall(), columns=['pollution_id', 'sensor_id', 'pm25', 'pm10', 'temp_celsius', 'date'])
df

,pollution_id,sensor_id,pm25,pm10,temp_celsius,date
0,1,34641,37.2,44.5,29.0,2023/03/09 18:13:32
1,2,37747,17.6,23.9,32.0,2023/03/09 18:13:33
2,3,38537,41.2,54.3,31.0,2023/03/09 18:13:34
3,4,143372,34.9,43.8,32.0,2023/03/09 18:13:35
4,5,39355,35.4,46.6,28.0,2023/03/09 18:13:35
...,...,...,...,...,...,...
91,92,36709,33.5,37.0,30.0,2023/03/09 18:37:04
92,93,33117,1.2,1.8,31.0,2023/03/09 18:37:04
93,94,35011,30.0,34.4,31.0,2023/03/09 18:37:05
94,95,34969,30.7,35.7,30.0,2023/03/09 18:37:06


In [12]:
# Fetch data from sensors
conn = sqlite3.connect('air-pollution.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          sensor_id, municipio, estado FROM sensors;
          ''')

df = pd.DataFrame(c.fetchall(), columns=['sensor_id', 'municipio', 'estado'])
df

,sensor_id,municipio,estado
0,33117,San Pedro Garza García,Nuevo Leon
1,34499,San Pedro Garza García,Nuevo Leon
2,34511,San Pedro Garza García,Nuevo Leon
3,34569,San Pedro Garza García,Nuevo Leon
4,34641,San Pedro Garza García,Nuevo Leon
5,34969,San Pedro Garza García,Nuevo Leon
6,35011,San Pedro Garza García,Nuevo Leon
7,36709,San Pedro Garza García,Nuevo Leon
8,37253,San Pedro Garza García,Nuevo Leon
9,37255,San Pedro Garza García,Nuevo Leon


temp_celsius